# Test Phase Transition at Finite Temperature for Different Benchmark Points

In [1]:
import numpy as np
from cosmoTransitions import generic_potential as gp
from cosmoTransitions.finiteT import Jb_spline as Jb
from cosmoTransitions.finiteT import Jf_spline as Jf

## Constants
- $m_H$ = 125. GeV
- $v_{EW}$ = 174. GeV

In [4]:
GF = 1.16637e-05
v = 1/(np.sqrt(2*np.sqrt(2)*GF))
mHSM = 125.13

## Transfer physical parameters to bare parameters
$$
\begin{align*}
\mu_H^2 &= \frac{1}{2}(m_h^2 \cos^2 \theta + m_S^2 \sin^2 \theta)\\
\mu_S^2 &= m_h^2 \sin^2 \theta + m_S^2 \cos^2 \theta\\
A &= \frac{(m_h^2 - m_S^2)\sin \theta \cos \theta}{\sqrt{2}v}\\
\lambda &= \frac{m_h^2 \cos^2 \theta + m_S^2 \sin^2 \theta}{4 v^2}
\end{align*}
$$


In [5]:
def muH2(mS, sintheta):
    """\mu_H square"""
    return 0.5*(mHSM**2 * (1-sintheta**2) + mS**2 * sintheta**2)

def muS2(mS, sintheta):
    """\mu_S square"""
    return sintheta**2 * mHSM**2 + (1 - sintheta**2) * mS**2

def A(mS, sintheta):
    """A parameter"""
    nominator = (mHSM**2 - mS**2) * sintheta * np.sqrt(1-sintheta**2)
    denominator = np.sqrt(2) * v
    return nominator/denominator

def lm(mS, sintheta):
    """\lambda parameter"""
    nominator = (1 - sintheta**2)*mHSM**2 + sintheta**2 * mS**2
    return nominator/(4*v**2)

## Define effective potential

In [13]:
class model(gp.generic_potential):
    def init(self, mS, sintheta):
        self.Ndim = 2
        self.Tmax = 100
        self.mS = mS
        self.sintheta = sintheta
        self.lm = lm(self.mS, self.sintheta)
        self.A = A(self.mS,self.sintheta)
        self.muH2 = muH2(self.mS,self.sintheta)
        self.muS2 = muS2(self.mS,self.sintheta)
        self.g = 0.65
        self.gY = 0.36
        self.yt = 0.9945
        self.D = (3*self.g**2 + self.gY**2 + 4*self.yt**2)/16.
        self.E = (2*self.g**3+(self.g**2 + self.gY**2)**(3/2))/(48*np.pi)
        self.cs = 1./3
        self.Deff = self.D - self.cs * self.A**2/(4.*self.muS2)
        self.lmeff = self.lm - self.A**2/(2*self.muS2)
        self.T0 = np.sqrt(0.5*self.muH2 - v**2 * self.A**2 /(2*self.muS2))/np.sqrt(self.D - self.cs*self.A**2/(4*self.muS2))
        self.Tc = self.T0*np.sqrt((self.Deff * self.lmeff)/(-self.E**2 + self.Deff*self.lmeff))
        self.strength = 2*self.E/self.lmeff

    def Vtot1d(self, X, T, include_radiation = True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T*T) + 1e-100
        phi1 = X[...,0]
        y = self.Deff * T2 * phi1**2 - (0.5*self.muH2 - 0.5 * v**2 * self.A**2 / (self.muS2))*phi1**2
        y += - self.E * T * phi1 **3
        y += 0.25 * self.lmeff * phi1**4
        return y



    def Vtot(self, X, T, include_radiation=True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T*T)+1e-100
        phi1 = X[...,0]
        phi2 = X[...,1]
        y = self.D * T2 * phi1**2 - 0.5 * self.muH2 * phi1**2
        y += - self.E * T * phi1**3
        y += 0.25 * self.lm * phi1**4
        y += 0.5*self.muS2*phi2**2 - 0.5 * self.A * (phi1**2 + self.cs * T2 - 2 * v**2)*phi2
        return y



## Test Phase Transition Computation

In [14]:
m=model(5, 0.17)

In [15]:
m.findMinimum(T=0.)

array([ 2.46220567e+02, -2.61527349e-05])

In [21]:
m.Tc

25.78022696942942

In [20]:
m.findAllTransitions()

Tracing phase starting at x = [ 2.46220569e+02 -7.94546948e-07] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0
.....................................
Tracing phase starting at x = [ 210.05295023 -183.0103545 ] ; t = 16.88146037802462
Tracing minimum down
traceMinimum t0 = 16.8815
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

[{'low_vev': array([  62.48637619, -630.21658423]),
  'high_vev': array([ 1.73468790e-05, -6.73768609e+02]),
  'Tnuc': 25.66366127971011,
  'low_phase': 1,
  'high_phase': 2,
  'instanton': fullTunneling_rval(profile1D=Profile1D(R=array([0.00000000e+00, 5.14105431e-05, 9.62364172e-03, 1.91958729e-02,
         2.87681041e-02, 3.83403353e-02, 4.79125664e-02, 5.74847976e-02,
         6.70570288e-02, 7.66292600e-02, 8.62014911e-02, 9.57737223e-02,
         1.05345954e-01, 1.14918185e-01, 1.24490416e-01, 1.34062647e-01,
         1.43634878e-01, 1.53207109e-01, 1.62779341e-01, 1.72351572e-01,
         1.81923803e-01, 1.91496034e-01, 2.01068265e-01, 2.10640496e-01,
         2.20212728e-01, 2.29784959e-01, 2.39357190e-01, 2.48929421e-01,
         2.58501652e-01, 2.68073884e-01, 2.77646115e-01, 2.87218346e-01,
         2.96790577e-01, 3.06362808e-01, 3.15935039e-01, 3.25507271e-01,
         3.35079502e-01, 3.44651733e-01, 3.54223964e-01, 3.63796195e-01,
         3.73368427e-01, 3.82940658e-01, 

In [22]:
m.prettyPrintTnTrans()

First-order transition at Tnuc = 25.66
High-T phase:
  key = 2; vev = [ 1.73468790e-05 -6.73768609e+02]
Low-T phase:
  key = 1; vev = [  62.48637619 -630.21658423]
Pressure difference = 5888 = (8.76)^4
Energy difference = 5888 = (8.76)^4
Action = 3610
Action / Tnuc = 140.668



In [35]:
Tn = m.TnTrans[0]['Tnuc']

In [36]:
m.getPhases()

Tracing phase starting at x = [ 2.46220569e+02 -7.94546948e-07] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0
.....................................
Tracing phase starting at x = [ 210.05295023 -183.0103545 ] ; t = 16.88146037802462
Tracing minimum down
traceMinimum t0 = 16.8815
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

{0: Phase(key=0, X=[[246.2 -7.945e-07], ..., [244.4 -0.0006787]], T=[0, ..., 16.78], dXdT=[[-0 -0], ..., [-0 -0]],
 1: Phase(key=1, X=[[246.2 -0.02073], ..., [36.78 -658.7]], T=[7.473, ..., 25.81], dXdT=[[-0 -0], ..., [-0 -0]],
 2: Phase(key=2, X=[[1.724e-05 -673.8], ..., [6.345e-06 -639]], T=[25.54, ..., 100], dXdT=[[-0 0], ..., [-0 0]]}

In [40]:
m.phases

{0: Phase(key=0, X=[[246.2 -7.945e-07], ..., [244.4 -0.0006787]], T=[0, ..., 16.78], dXdT=[[-0 -0], ..., [-0 -0]],
 1: Phase(key=1, X=[[246.2 -0.02073], ..., [36.78 -658.7]], T=[7.473, ..., 25.81], dXdT=[[-0 -0], ..., [-0 -0]],
 2: Phase(key=2, X=[[1.724e-05 -673.8], ..., [6.345e-06 -639]], T=[25.54, ..., 100], dXdT=[[-0 0], ..., [-0 0]]}